# ClaRet - Retinal Scan Classification

## Importing Libraries and Frameworks

In [1]:
import os 
import random
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow_addons.metrics import F1Score


print("Libraries imported")

Libraries imported


## Defining the master directory path

In [2]:
path = ("../input/kermany2018/oct2017/OCT2017 ")
os.listdir(path)

['val', '.DS_Store', 'test', 'train']

## Storing the path of the training, validation and testing directories

### Training Directory

In [3]:
train_dir = path + '/train'
os.listdir(train_dir)

['DRUSEN', 'CNV', 'NORMAL', 'DME']

### Validation Directory

In [4]:
val_dir = path + '/val'
os.listdir(val_dir)

['DRUSEN', 'CNV', 'NORMAL', '.DS_Store', 'DME']

### Testing Directory

In [5]:
test_dir = path + '/test'
os.listdir(test_dir)

['DRUSEN', 'CNV', 'NORMAL', '.DS_Store', 'DME']

## Defining The VGG19 model

In [6]:
vgg19 = tf.keras.applications.VGG19(
    include_top = False,
    weights = 'imagenet',
    input_tensor = None,
    input_shape = (150,150,3),
    pooling  = None,
    classes = 1000
)

80142336/80134624 [==============================] - 0s 0us/step


## Defining the Transfer Learning Model Architecture - VGG19

### Not letting VGG19 train all over again

In [7]:
vgg19.trainable = False
print("Trainable set to false")

Trainable set to false


### Defining the layers

In [8]:
model_vgg = tf.keras.models.Sequential([
    
    vgg19,
    
    tf.keras.layers.Conv2D(filters = 256, kernel_size = (3, 3), padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(units=1024,activation='relu'),
    
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(units=512,activation='relu'),
    
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(units=256,activation='relu'),
    
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(units=128,activation='relu'),
    
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation='relu'),
    
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(units = 4,activation = 'softmax')
])

## Model Compilation


### Defining Metrics

In [9]:
F1_Score = tfa.metrics.F1Score(num_classes=4, threshold=0.5)

In [10]:
vgg_metrics = ["accuracy", F1_Score]

### Compiling the full model by addidng optimiser, loss and metrics

In [11]:
model_vgg.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = vgg_metrics)
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 4, 256)         1179904   
_________________________________________________________________
re_lu (ReLU)                 (None, 4, 4, 256)         0         
_________________________________________________________________
dense (Dense)                (None, 4, 4, 1024)        263168    
_________________________________________________________________
dropout (Dropout)            (None, 4, 4, 1024)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 128)         1179776   
_________________________________________________________________
re_lu_1 (ReLU)               (None, 4, 4, 128)         0

## Rescaling Images and Defining generators

### Training Dataset

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150, 150), class_mode = 'categorical', batch_size = 500)

Found 83484 images belonging to 4 classes.


### Validation Dataset

In [13]:
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(val_dir, target_size = (150, 150), class_mode = 'categorical', batch_size = 16)

Found 32 images belonging to 4 classes.


### Testing Dataset

In [14]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size = (150, 150), class_mode = 'categorical', batch_size = 50)

Found 968 images belonging to 4 classes.


## Training the model

In [15]:
vgg_history = model_vgg.fit(
    train_generator,
    steps_per_epoch = (83484/500),
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = (32/16),
    max_queue_size=100,
    workers = 4 ,
    use_multiprocessing=True,
    verbose = 1)



Epoch 1/30
166/166 [==============================] - 365s 2s/step - loss: 0.4356 - accuracy: 0.5783 - f1_score: 0.3065 - val_loss: 0.2538 - val_accuracy: 0.7500 - val_f1_score: 0.6127
Epoch 2/30
166/166 [==============================] - 479s 3s/step - loss: 0.2231 - accuracy: 0.8230 - f1_score: 0.6856 - val_loss: 0.1085 - val_accuracy: 0.9062 - val_f1_score: 0.9051
Epoch 3/30
166/166 [==============================] - 484s 3s/step - loss: 0.1843 - accuracy: 0.8607 - f1_score: 0.7765 - val_loss: 0.0844 - val_accuracy: 0.9062 - val_f1_score: 0.9198
Epoch 4/30
166/166 [==============================] - 486s 3s/step - loss: 0.1620 - accuracy: 0.8795 - f1_score: 0.8134 - val_loss: 0.0537 - val_accuracy: 0.9688 - val_f1_score: 0.9686
Epoch 5/30
166/166 [==============================] - 486s 3s/step - loss: 0.1536 - accuracy: 0.8849 - f1_score: 0.8207 - val_loss: 0.0413 - val_accuracy: 1.0000 - val_f1_score: 1.0000
Epoch 6/30
166/166 [==============================] - 487s 3s/step - loss: 

## Testing the model

In [16]:
testing_model = model_vgg.evaluate(test_generator)


20/20 [==============================] - 21s 945ms/step - loss: 0.0686 - accuracy: 0.9628 - f1_score: 0.9628


## Compiling Results

### Storing accuracies

#### Storing the training accuracy

In [17]:
training_accuracy = (vgg_history.history['accuracy'][-1]*100)

#### Storing the validation accuracy

In [18]:
validation_accuracy = (vgg_history.history['val_accuracy'][-1]*100)

#### Storing the testing accuracy

In [19]:
testing_accuracy = testing_model[1]*100

## Storing Loss


#### Storing the Training Loss

In [20]:
training_loss = (vgg_history.history['loss'][-1]*100)

#### Storing the Validation Loss

In [21]:
validation_loss = (vgg_history.history['val_loss'][-1]*100)

#### Storing the Training Loss

In [22]:
testing_loss = testing_model[0]*100

## Storing F1 Scores

#### Storing the Training F1 Score

In [23]:
f1_score = (vgg_history.history['f1_score'][-1]*100)

#### Storing the Validation F1 Score

In [24]:
val_f1_score = (vgg_history.history['val_f1_score'][-1]*100)

#### Storing the Testing F1 Score

In [25]:
testing_f1_score = (testing_model[2])

## Compiled Result

### Accuracy

In [26]:
print("Training Accuracy : " + str(training_accuracy) + " %")
print("Validation Accuracy : " + str(validation_accuracy) + " %")
print("Testing Accuracy : " + str(testing_accuracy) + " %")

Training Accuracy : 95.72612643241882 %
Validation Accuracy : 96.875 %
Testing Accuracy : 96.28099203109741 %


### Loss

In [27]:
print("Training Loss : " + str(training_loss) + " %")
print("Validation Loss : " + str(validation_loss) + " %")
print("Testing Loss : " + str(testing_loss) + " %")

Training Loss : 5.998558923602104 %
Validation Loss : 12.220136821269989 %
Testing Loss : 6.864471733570099 %


### F1 Scores

In [28]:
print("F1 Score : " + str(f1_score))
print("Validation F1 Score : " + str(val_f1_score))
print("Testing F1 Score : " + str(testing_f1_score))

F1 Score : [97.99427  94.347534 85.68152  96.32771 ]
Validation F1 Score : [ 94.11765   93.333336 100.       100.      ]
Testing F1 Score : [0.9508841  0.97468346 0.9462366  0.9795082 ]
